# Advanced MPAS Analysis & Visualization with UXarray

<img src="path or URL to some visual here"
     width="30%"
     alt="MPAS advanced visual"
     align="right"
/>

### In this section, you'll learn:

* Utilizing ...

### Related Documentation

* [URL title](URL)
* 

### Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [URL title](URL) | Necessary OR Helpful?  | |

**Time to learn**: 30 minutes?

-----

## Import packages

In [ ]:
%%time 

# autoload external python modules if they changed
%load_ext autoreload
%autoreload 2

# add ../funcs to the current path
import sys, os
sys.path.append(os.path.join(os.getcwd(), "..")) 

# import modules
import warnings
import math

import cartopy.crs as ccrs
import geoviews as gv
import geoviews.feature as gf
import holoviews as hv
import hvplot.xarray
from holoviews import opts
import matplotlib.pyplot as plt

import s3fs

import geopandas as gp
import numpy as np
import uxarray as ux
import xarray as xr

## Configure visualization tools

In [ ]:
# hv.extension("bokeh")
# hv.extension("matplotlib")


# common border lines
coast_lines = gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale="50m")
state_lines = gf.states(projection=ccrs.PlateCarree(), line_width=1, line_color='gray', scale="50m")

## Load MPAS data
Depending on the network, the data loading process may take a few minutes

### Data paths to the JetStream2 and S3 objects

In [ ]:
%%time

jetstream_url = 'https://js2.jetstream-cloud.org:8001/'
fs = s3fs.S3FileSystem(anon=True, asynchronous=False,client_kwargs=dict(endpoint_url=jetstream_url))
conus12_path = 's3://pythia/mpas/conus12km'

grid_url=f"{conus12_path}/conus12km.invariant.nc_L60_GFS"
bkg_url=f"{conus12_path}/bkg/mpasout.2024-05-06_01.00.00.nc"
ana_url=f"{conus12_path}/ana/mpasout.2024-05-06_01.00.00.nc"

grid_file = fs.open(grid_url)
ana_file = fs.open(ana_url)
bkg_file = fs.open(bkg_url)

### Open UXarray datasets

In [ ]:
%%time 

uxds_a = ux.open_dataset(grid_file, ana_file)
uxds_b = ux.open_dataset(grid_file, bkg_file)

In [ ]:
uxvar = uxds_a['theta'] - 273.15   ## Kelvin to Celsius

Also, for simplicity, let's focus at the vertical level and time indices of 0. 

In [ ]:
i_lev = 0   # `nVertLevels` index
i_time = 0  # `Time` index

## Vertical cross section

In [ ]:
# %%time

# tmp = vslice_contour(uxvar, lat=42.63, clevels=10)
# display(tmp)

Let us use UXarray's vertical cross-section function to get a cross-section over a great circle arc:

In [ ]:
%%time

start_point = (-120, 20)
end_point = (-70, 50)

cross_section_gca = uxvar.cross_section(start=start_point, end=end_point, steps=100)

UXarray's cross-section returns an `xarray.DataArray` that can then be plotted:

In [ ]:
cross_section_gca.isel(Time=0).transpose().plot.contourf()